In [1]:
import numpy as np
import pandas as pd
from function.function_black import vectorized_premium
from function.function_select import select_index_batch

expiry = 'NearbyMonth'
IV_Data = pd.read_csv('./../../Data/Organized/IV/{}Complete.csv'.format(expiry), 
                      encoding='Big5', index_col=False)
IV_matrix = np.array(IV_Data)
idx_IV_matrix = np.arange(len(IV_matrix))

column_call_put_name = '買賣權'
column_call_put_index = IV_Data.columns.get_loc(column_call_put_name)

#買權資料在原本資料的哪幾列
rows_call = idx_IV_matrix[np.equal(IV_matrix[:, column_call_put_index], 'call')]
#買權資料的交易日期、到期日期、履約價
DateK_call = IV_matrix[rows_call, :3]

#賣權資料在原本資料的哪幾列
rows_put = idx_IV_matrix[np.equal(IV_matrix[:, column_call_put_index], 'put')]
#賣權資料的交易日期、到期日期、履約價
DateK_put = IV_matrix[rows_put, :3]

#找出擁有相同交易日期、到期日期、履約價的買權與賣權，找出匹配的資料分別位於買權與賣權資料的第幾列
# idx_call_filter即匹配資料為於買權的索引，idx_put_filter即匹配資料為於賣權的索引
#匹配資料代表著：同一交易日到期日與履約價，其同時有買權與賣權
idx_call_filter, idx_put_filter = select_index_batch(DateK_call, DateK_put, 10000, 10000, axis=1)

#匹配資料位於原資料中、為買權((存在同交易日到期日履約價之賣權))的位於原資料哪幾列
rows_call_filter = rows_call[idx_call_filter]
#匹配資料位於原資料中、為賣權((存在同交易日到期日履約價之買權))的位於原資料哪幾列
rows_put_filter = rows_put[idx_put_filter]
#匹配資料位於原資料中、為賣權(存在同交易日到期日履約價之買權)的哪幾列的值為False，其餘為True
not_put_filter = np.ones(len(IV_matrix), dtype=bool)
not_put_filter[rows_put_filter] = False







column_names_reserve = ['交易日期', '到期日期', '履約價', '期貨成交量', \
        '期貨開盤價','期貨最高價', '期貨最低價', '期貨收盤價', '期貨結算價', '到期天數', '無風險利率']
column_index_reserve = [IV_Data.columns.get_loc(col) for col in column_names_reserve]

IV_matrix_reserve = IV_matrix[not_put_filter, :]
IV_matrix_reserve = IV_matrix_reserve[:, column_index_reserve]



volume = (IV_matrix[:, IV_Data.columns.get_loc('成交量')]).copy() #成交量
#買權的成交量向量
volume_call = volume[rows_call]

#賣權的成交量向量
volume_put = volume[rows_put]

    
#買權成交量的向量，且存在與其相同交易日到期日履約價的賣權
volume_call_filter = volume[rows_call_filter]
#賣權成交量的向量，且存在與其相同交易日到期日履約價的買權
volume_put_filter = volume[rows_put_filter]
#volume_call_filter 與 volume_put_filter 有相同交易日到期日與履約價，只是買賣權不同 

#買賣權成交量相加
volume_sum_filter = volume_call_filter + volume_put_filter
volume_final = volume.copy()

#將那些「存在相同交易日到期日履約價之賣權」的買權的成交量改成買賣權成交量相加
volume_final[rows_call_filter] = volume_sum_filter 
#刪除掉「存在相同交易日到期日履約價之買權」的賣權的資料
volume_final = volume_final[not_put_filter] 

#將那些「存在相同交易日到期日履約價之賣權」的買權的改成'call and put'
c_or_p = (IV_matrix[:, IV_Data.columns.get_loc('買賣權')]).copy()
c_or_p[rows_call_filter] = 'call and put'
#刪除掉「存在相同交易日到期日履約價之買權」的賣權的資料
c_or_p = c_or_p[not_put_filter] 



In [3]:
price_types = ['收盤價', '結算價']

IV_final = np.zeros((2, len(volume_final)))
for i in range(len(price_types)):

    iv = (IV_matrix[:, IV_Data.columns.get_loc('隱含波動率({})'.format(price_types[i]))]).copy() 



    #買權的隱含波動率向量
    iv_call = iv[rows_call] 
    #賣權的隱含波動率向量
    iv_put = iv[rows_put]

    #調整後的買權隱含波動率的向量，且存在與其相同交易日到期日履約價的賣權
    iv_call_filter = iv[rows_call_filter]
    #調整後的賣權隱含波動率的向量，且存在與其相同交易日到期日履約價的買權
    iv_put_filter = iv[rows_put_filter]

    #iv_call_filter 與 iv_put_filter 有相同交易日到期日與履約價，只是買賣權不同 


    #依照成交量為基礎，將擁有相同履約價、交易日、到期日的買權與賣權的隱含波動率取個加權平均
    weight_iv_call = volume_call_filter / volume_sum_filter
    weight_iv_put = volume_put_filter / volume_sum_filter
    weight_avg_iv = weight_iv_call*iv_call_filter + weight_iv_put*iv_put_filter
    iv_final = iv.copy()


    #將那些「存在相同交易日到期日履約價之賣權」的買權的隱含波動率改成加權平均的隱含波動率
    iv_final[rows_call_filter] = weight_avg_iv
    #刪除掉「存在相同交易日到期日履約價之買權」的賣權的資料
    iv_final = iv_final[not_put_filter] 

    IV_final[i] = iv_final

CPVIV_matrix_new = (np.vstack((c_or_p, volume_final, IV_final))).T
IV_matrix_new= np.hstack((IV_matrix_reserve, CPVIV_matrix_new))


column_names_new = np.array(['買賣權', '成交量', '隱含波動率(收盤價)', '隱含波動率(結算價)'])
column_name_all = np.hstack((np.array(column_names_reserve), column_names_new))
IV_Data_new = pd.DataFrame(data=IV_matrix_new, columns=column_name_all)
IV_Data_new.to_csv('./../../Data/Organized/MergeIV/{}.csv'.format(expiry), encoding='Big5',\
                   index = False)


print('Finish !!')

Finish !!
